In [ ]:
from numpy import random
from math import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from numpy import *
from imblearn import *
import seaborn as sns

In [ ]:
majority_c1= 0.5
majority_std_deviation= 0.2
majority_c2= 1
majority_size=4500

percentage= 60
minority_std_deviation= 0.1
minority_c1= majority_c1 + (1 - percentage/100)
minority_c2= 1
minority_size=500

while True:
  x_1=[]
  y_1=[]
  for _ in range(0,majority_size):
    x_1.append(random.normal()*majority_std_deviation + majority_c1)
    y_1.append(random.normal()*majority_std_deviation + majority_c2)
  min_x1= min(x_1)
  min_y1= min(y_1)
  center_x= sum(x_1)/len(x_1)
  center_y= sum(y_1)/len(y_1)
  max_x1=max(x_1)
  max_y1=max(y_1)

  x_2=[]
  y_2=[]
  for _ in range(0,minority_size):
    x_2.append(random.normal()*minority_std_deviation + minority_c1)
    y_2.append(random.normal()*minority_std_deviation + minority_c2)
  min_x2= min(x_2)
  min_y2= min(y_2)

  max_1= max(x_1)
  min_2= min(x_2)
  max_2= max(x_2)

  percentage_overlapping= (max_1-min_2)/max_2*100
  if percentage_overlapping <= percentage + 0.1 and percentage_overlapping >= percentage - 0.1:
    break
majority_data = pd.DataFrame({'x': x_1,'y': y_1})
plt.plot(x_1, y_1, 'bo')
plt.show()
print(percentage_overlapping)
minority_data = pd.DataFrame({'x': x_2,'y': y_2})
plt.plot(x_2, y_2, 'ro')
plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(x_1, y_1, s=10, c='b', marker="s", label='Majority')
ax1.scatter(x_2,y_2, s=10, c='r', marker="o", label='Minority')
plt.legend(loc='upper left');
plt.show()

In [ ]:
Dataset= majority_data.append(minority_data, ignore_index=True)
majority_y=['Positive'] * 4500
minority_y=['Negative'] * minority_size
y=majority_y + minority_y
X_train, X_test, y_train, y_test = train_test_split(Dataset, y, test_size=0.33, random_state=1)

In [ ]:
X_train

In [ ]:
l1=X_train['x'].to_list()
l2=X_train['y'].to_list()
train=[]
for x,y in zip(l1,l2):
    train.append([x,y])

In [ ]:
grid = {'C': [0.1,0.5,1,10],
              'gamma': [0.1,1,10,50,100],
              'kernel': ['rbf']}

In [ ]:
rf_cv = GridSearchCV(estimator=SVC(), param_grid=grid, cv= 5)
rf_cv.fit(X_train, y_train)

In [ ]:
params=rf_cv.best_params_

In [ ]:
!pip install libsvm

In [ ]:
import libsvm.svmutil as svm

In [ ]:
X_train=list(X_train)
y_train=list(y_train)

In [ ]:
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
y_train = LE.fit_transform(y_train)
y_train=list(y_train)

In [ ]:
print(params)

In [ ]:
# param = '-c ' + str(params['C']) + ' -t ' + str(2) + ' -g '+ str(params['gamma'])

In [ ]:
param = '-c ' + str(1) + ' -t ' + str(2) + ' -g '+ str(10)

In [ ]:
# param = '-c ' + str(0.2) + ' -t ' + str(2) + ' -g '+ str(50)

In [ ]:
model = svm.svm_train(y_train, train, param)

In [ ]:
w = np.matmul(np.array(train)[np.array(model.get_sv_indices()) - 1].T, model.get_sv_coef())
b = -model.rho.contents.value
if model.get_labels()[1] == -1:
    w = -w
    b = -b

print(w)
print(b)

In [ ]:
rho = model.rho[0] 
svs = model.get_SV() 
sv_coeffs = model.get_sv_coef()

In [ ]:
indices=model.get_sv_indices()

In [ ]:
print(rho)
print(svs)
print(sv_coeffs)

In [ ]:
sv_coeffs1=[]
for i in range(0, len(sv_coeffs)):
    sv_coeffs1.append(sv_coeffs[i][0])

In [ ]:
len(indices)

In [ ]:
train[1]

In [ ]:
supportvectors=[]
for i in range(0,len(indices)):
  supportvectors.append([train[i][0],train[i][1]])

In [ ]:
w1=[]
for i in range(0,len(indices)):
  w1.append([supportvectors[i][0]*sv_coeffs1[i],supportvectors[i][1]*sv_coeffs1[i]])

In [ ]:
import math

wsum=0
for i in range(0, len(w1)):
  wsum=wsum+ (math.pow(w1[i][0],2)+ math.pow(w1[i][1],2))

wsum=math.sqrt(wsum)

In [ ]:
x_3=[]
y_3=[]
increased_minority_size= Counter(y_train)[1] - Counter(y_train)[0]
while len(x_3)!=increased_minority_size:
  x= random.normal()*minority_std_deviation + minority_c1
  y= random.normal()*minority_std_deviation + minority_c2

  tmpdist=abs(x)/(abs(wsum)- rho)
# Probability next 2 lines
# instanceprobability= 1/tmpdist * 0.02
# if(instanceprobability > random.normal()):

# Without probability 
  if(tmpdist > random.normal()):

    x_3.append(x)
    y_3.append(y)
minority_data_increased = pd.DataFrame({'x': x_3,'y': y_3})


In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(x_1, y_1, s=10, c='b', marker="s", label='Majority')
ax1.scatter(x_2,y_2, s=10, c='r', marker="o", label='Minority')
ax1.scatter(x_3,y_3, s=10, c='g', marker="o", label='Oversampled')
plt.legend(loc='upper left');
plt.show()

In [ ]:
tmp = {'x': x_1 + x_2, 'y': y_1 + y_2}
X_train = pd.DataFrame(data=tmp)
y_train= ["Positive"] * len(x_1) + ["Negative"] * len(x_2)

In [ ]:
print(len(y_train))

In [ ]:
center_x= sum(x_3)/len(x_3)
center_y= sum(y_3)/len(y_3)

print(center_x)
print(center_y)


x_1=[]
y_1=[]
x_2=[]
y_2=[]
for x,y,val in zip(X_train['x'],X_train['y'],y_train):
  if val == "Positive":
    x_1.append(x)
    y_1.append(y)
  else:
    x_2.append(x)
    y_2.append(y)

majority_data = pd.DataFrame({'x': x_1,'y': y_1})
minority_data = pd.DataFrame({'x': x_2,'y': y_2})
minority_data= minority_data.append(minority_data_increased, ignore_index=True)

In [ ]:
Dataset= majority_data.append(minority_data, ignore_index=True)
majority_y=["Positive"] * majority_data.shape[0]
minority_y=["Negative"] * minority_data.shape[0]
y=majority_y + minority_y
X_train, y_train = Dataset,y

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(len(y_train))
print(len(y_test))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

In [ ]:
y_pred  =  classifier.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test,y_pred)
print(ac)
print(cm)

In [ ]:
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test) 
cm = confusion_matrix(y_test, y_pred)
print ("Accuracy : ", accuracy_score(y_test, y_pred))
print(cm)

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [ ]:
y_pred = classifier.predict(X_test) 
cm = confusion_matrix(y_test, y_pred)
print ("Accuracy : ", accuracy_score(y_test, y_pred))
print(cm)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(y_test, y_pred, average='weighted')

In [ ]:
print(percentage_overlapping)